## Notebook to compute total demand by TAZ for the non-motorized mode

In [ ]:
# Notebook to compute total demand by TAZ for the non-motorized mode

import openmatrix as omx
import numpy as np
import pandas as pd
import geopandas as gp
import matplotlib.pyplot as plt
import bokeh
import xarray as xr
import hvplot.pandas
import hvplot.xarray

In [ ]:
%matplotlib notebook

### User input required: Specify paths to input and output directories in config.py file

In [ ]:
%run "S:/jupyter_notebooks/config.py"

## User input required: specify path to root directory of scenario to use

In [ ]:
# Specifiy scenario to use for the current run of this notebook:
# 
home_dir = base_scenario_dir
# 
# Supply name of CSV output file for tabular results generated by this notebook:
#
csv_output_fn = 'taz_nm_report_base.csv'

## User input required: Specify name of CSV output file

In [ ]:
# Supply name of CSV output file for tabular results generated by this notebook:
#
csv_output_fn = 'taz_nm_report_base.csv'

In [ ]:
taz_shapefile_base_dir = reference_data_dir + 'canonical_TAZ_shapefile/'

In [ ]:
# trip_tables directory - this really "should" be a subdirectory of the base directory, but is isn't currently.
# The real McCoy - where things should go, and will eventually go
tt_dir = home_dir + 'out/'

In [ ]:
# Function to open OMX trip_tables files
def open_trip_tables(tt_dir):
    tt_am = tt_dir + 'AfterSC_Final_AM_Tables.omx'
    tt_md = tt_dir + 'AfterSC_Final_MD_Tables.omx'
    tt_pm = tt_dir + 'AfterSC_Final_PM_Tables.omx'
    tt_nt = tt_dir + 'AfterSC_Final_NT_Tables.omx'
    tt_omxs = { 'am' : omx.open_file(tt_am,'r'),
                'md' : omx.open_file(tt_pm,'r'),
                'pm' : omx.open_file(tt_pm,'r'),
                'nt' : omx.open_file(tt_nt,'r') 
              }   
    return tt_omxs
# end_def open_trip_tables

In [ ]:
# Function to load trip tables as NumPy arrays from OMX files
def load_trip_tables(tt_omxs, modes):
    periods = ['am', 'md', 'pm', 'nt']
    retval  = {'am' : {}, 'md' : {}, 'pm' : {}, 'nt' : {} }
    for period in periods:
        for mode in modes:
            temp = tt_omxs[period][mode]
            retval[period][mode] = np.array(temp)
        # end_for
    # end_for
    return retval
# end_def 

## Load trip tables for non-motorized mode

In [ ]:
# *** Load trip tables for non-motorized mode
# 
tt_omxs = open_trip_tables(tt_dir)
temp = load_trip_tables(tt_omxs, ['Bike','Walk'])

In [ ]:
# Mapping from TAZ-ID to OMX index for the 4 periods (these *should* be the same)
taz_to_omxid_am = tt_omxs['am'].mapping('ID')
taz_to_omxid_am = tt_omxs['md'].mapping('ID')
taz_to_omxid_pm = tt_omxs['pm'].mapping('ID')
taz_to_omxid_nt = tt_omxs['nt'].mapping('ID')

In [ ]:
# We'll assume that the mapping from TAZ ID to OMX ID doesn't vary by time period.
# We'll use the AM mapping as _the_ mapping for all time periods, pending confirmation.
# 
# TBD: Insert "sanity check" that the 4 mappings on "ID" are identical.
#
taz_to_omxid = taz_to_omxid_am

In [ ]:
# Compute total Bike, Walk and total nonmotorized demand from each TAZ
bike_temp = temp['am']['Bike'] + temp['md']['Bike'] + temp['pm']['Bike'] + temp['nt']['Bike']
bike_demand = bike_temp.sum(axis=1)
walk_temp = temp['am']['Walk'] + temp['md']['Walk'] + temp['pm']['Walk'] + temp['nt']['Walk']
walk_demand = walk_temp.sum(axis=1)
total_nm_demand = bike_demand + walk_demand

In [ ]:
# Build dataframes of Bike, Walk, and total nonmotorized demand from each TAZ
# Set each data frame's index to the omxid of each row, i.e., its index
bike_df = pd.DataFrame(bike_demand, columns=['bike'])
bike_df['omxid'] = bike_df.index
bike_df.set_index('omxid')
walk_df = pd.DataFrame(walk_demand, columns=['walk'])
walk_df['omxid'] = walk_df.index
walk_df.set_index('omxid')
nm_df = pd.DataFrame(total_nm_demand, columns=['nm_total'])
nm_df['omxid'] = nm_df.index
nm_df.set_index('omxid')

In [ ]:
# Join the 3 dataframes into a single data frame
temp_df = pd.merge(left=nm_df, right=bike_df, on="omxid")
total_nm_trips_df = pd.merge(temp_df, walk_df, on="omxid")

In [ ]:
# Load the candidate canonical TAZ shapefile as a geopands dataframe.
# N.B. Use shapefile in WGS84 SRS.
#
taz_shapefile = taz_shapefile_base_dir + 'candidate_CTPS_TAZ_STATEWIDE_2019_wgs84.shp'
taz_gdf = gp.read_file(taz_shapefile)
taz_gdf.set_index("id")

In [ ]:
# Add a 'omxid' column to the TAZ geodataframe, in prep for joining with the total trips dataframes.
# ==> This also can be done earlier.
taz_gdf['omxid'] = taz_gdf.apply(lambda row: taz_to_omxid[row.id], axis=1)

In [ ]:
# Join the shapefile geodataframe to the total trips dataframe on 'omxid'
joined_df = taz_gdf.join(total_nm_trips_df.set_index('omxid'), on='omxid')

## Export report output to CSV file

In [ ]:
# Export the useful columns of data in the 'joined_df' dataframe as a CSV file
fq_output_fn = sandbox_dir + csv_output_fn
joined_df.to_csv(fq_output_fn, sep=',', columns=['id', 'town', 'state', 'nm_total', 'bike', 'walk'])

## Make static and interactive maps of results

In [ ]:
# Make a static map of total nonmotorized trips by origin TAZ
joined_df.plot("nm_total", figsize=(10.0,8.0), cmap='plasma', legend=True)
plt.title('Total Non-motorized Trips by Origin TAZ')
plt.show()

In [ ]:
# Make an interactive map of the above
joined_df.hvplot(c='nm_total', 
                 geo=True, 
                 hover_cols=['id', 'town', 'nm_total', 'bike', 'walk'], 
                 clabel='Total Non-motorized Trips', 
                 cmap='plasma',
                 frame_height=500).opts(title='Total Non-motorized Trips by Origin TAZ')